In [2]:
import requests
import pandas as pd
from datetime import datetime, timezone
import json
import uuid
from google.cloud import storage
import os



dump the data news_snetiment into gcs bucket

develop modular way dump the pull into gcp -> see alphaverseScrape.ipynb

change bucket to us iowa to match with rafas mongo db


In [3]:

#######################
#update filepath and credentials
#####################
# set the environment variable directly in the script
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "file path to credentials need to grant roles"


In [4]:

# GCS Bucket name and initialize storage client

#change bucket name
####################
bucket_name = "alphaverse2020"
####################
storage_client = storage.Client()

# check if the bucket is accessible
try:
    bucket = storage_client.get_bucket(bucket_name)
    print(f"Successfully connected to bucket: {bucket_name}")
except Exception as e:
    print(f"Error accessing bucket: {e}")
    raise

Successfully connected to bucket: alphaverse2020


In [5]:
# dump local data /Users/catalinabartholomew/Documents/msdsMac/springMod1/DistDataSys/project/sentiment/news_sentiment_data_20250220_115715.json
# into gcp bucket


def upload_local_json_to_gcs(file_path, bucket_name, destination_prefix="news_sentiment"):
    """Uploads a local JSON file to GCS."""
    try:

        file_name = os.path.basename(file_path)
        
        # initialize the GCS client and bucket
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        
        # create blob with destination path
        destination_blob_name = f"{destination_prefix}/{file_name}"
        blob = bucket.blob(destination_blob_name)
        
        # upload the file
        blob.upload_from_filename(file_path)
        
        print(f"Successfully uploaded {file_name} to GCS")
        print(f"File path: gs://{bucket_name}/{destination_blob_name}")
        
    except Exception as e:
        print(f"Error uploading to GCS: {e}")
        raise

# upload the specific JSON file
file_path = "/Users/catalinabartholomew/Documents/msdsMac/springMod1/DistDataSys/project/sentiment/news_sentiment_data_20250220_115715.json"
upload_local_json_to_gcs(file_path, bucket_name)



Successfully uploaded news_sentiment_data_20250220_115715.json to GCS
File path: gs://alphaverse2020/news_sentiment/news_sentiment_data_20250220_115715.json


In [ ]:

# def upload_to_gcs(data, bucket_name, prefix="news_sentiment"):
#     """Uploads JSON data to GCS with a unique timestamped filename."""
#     try:
#         timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")  # Fixed datetime reference
#         unique_id = uuid.uuid4().hex[:6]
#         file_name = f"{prefix}_{timestamp}_{unique_id}.json"

#         json_data = json.dumps(data, indent=2)
        
#         bucket = storage_client.bucket(bucket_name)
#         blob = bucket.blob(f"news_sentiment/{file_name}")
#         blob.upload_from_string(json_data, content_type="application/json")

#         # verify upload
#         if blob.exists():
#             print(f"Successfully uploaded {file_name} with {len(data)} articles to GCS")
#             print(f"File path: gs://{bucket_name}/news_sentiment/{file_name}")
#         else:
#             print(f"Upload may have failed for {file_name}")
            
#     except Exception as e:
#         print(f"Error uploading to GCS: {e}")
#         raise


# mag7_tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "META", "TSLA"]

# # api_key = "7EX67VQNOOXOBWKE"
# api_key = "ZU0OJF3J90D08G85"
# base_url = "https://www.alphavantage.co/query"


# all_articles = []

# # loop through tickers to get news articles, or statements
# for ticker in mag7_tickers:
#     params = {
#         "function": "NEWS_SENTIMENT",
#         "tickers": ticker,  # query one ticker at a time
#         "time_from": "20200101T0000",  
#         "limit": 1000,  
#         "sort": "LATEST",
#         "apikey": api_key
#     }

#     response = requests.get(base_url, params=params)
#     data = response.json()

#     # check if the response contains 'feed' key
#     if "feed" in data and data["feed"]:
#         all_articles.extend(data["feed"])
#     else:
#         print(f"No articles found for ticker: {ticker}")

# # convert to DataFrame
# df = pd.DataFrame(all_articles)

# # drop duplicates based on the URL
# df.drop_duplicates(subset=["url"], inplace=True)

# # debugging : check if articles are found and print the number
# print(f"Total articles found: {len(all_articles)}")
# print(f"Unique articles after deduplication: {len(df)}")

# # if articles are found, upload to GCS
# if not df.empty:
#     batch_size = 500
#     for i in range(0, len(df), batch_size):
#         batch = df.iloc[i:i + batch_size].to_dict(orient="records")
#         print(f"Uploading batch {i // batch_size + 1} with {len(batch)} articles...")
#         upload_to_gcs(batch, bucket_name)
# else:
#     print("No articles to upload.")


In [6]:
#check bucket contents

bucket = storage_client.bucket(bucket_name)
blobs = bucket.list_blobs(prefix="news_sentiment/")
print("\nFiles in bucket:")
for blob in blobs:
    print(f"- {blob.name}")


Files in bucket:
- news_sentiment/news_sentiment_data_20250220_115715.json
